In [1]:
import torch
import pandas as pd 
import numpy as np
from xml.dom import minidom
import os

import re

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel


In [34]:
def get_text(filename = 'semcor/br-a01.xml', omit = False, vocabulary = False):
    mydoc = minidom.parse(filename)
    items = mydoc.getElementsByTagName('word')
    text = []
    sense = []
    for elem in items:  
        if elem.attributes['text'].value in vocabulary:
            try:
                if elem.attributes['sense']:
                    text.append('[MASC]')
                    sense.append((elem.attributes['text'].value, elem.attributes['sense'].value))
            except Exception:
                text.append(elem.attributes['text'].value)
                
    '''    if omit:
        for elem in items: 
            if elem.attributes['text'].value in vocabulary:
                try:
                    if elem.attributes['sense']:
                        text.append('[MASC]')
                        sense[elem.attributes['text'].value] = elem.attributes['sense'].value
                except Exception:
                    text.append(elem.attributes['text'].value)'''
                    
    return (text, sense) 

In [30]:
filenames = []
directory = 'semcor/'
for filename in os.listdir(directory):
    if filename.endswith(".xml"): 
        filenames.append(os.path.join(directory, filename))
        continue
    else:
        continue

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
Vocabulary = set(tokenizer.vocab.keys())
model = BertModel.from_pretrained('bert-large-cased')

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [35]:
docs = []
i = 0
for filename in filenames:
    i+=1
    #print(i)
    docs.append(get_text(filename, True, Vocabulary))

In [12]:
tokens_tensor = torch.tensor(tokenizer.convert_tokens_to_ids(docs[0][0]))[None, :]
tokens_tensor

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (2167 > 512). Running this sequence through BERT will result in indexing errors


tensor([[ 2857,  1254,   117,  ..., 10638,   107,   119]])

In [45]:
def get_embedding_text(text, model, num_layer, idx, window):
    leftBound = max(0, idx-window)
    rightBound = min(idx+window, len(text))
    sentences = text[leftBound:rightBound]
    #sentences.remove('[MASC]')
    sentences = list(filter(lambda a: a != '[MASC]', sentences))
    tokens_tensor = torch.tensor(tokenizer.convert_tokens_to_ids(sentences))[None, :]
    with torch.no_grad():
        encoded_layers,_  = model(tokens_tensor)
    return encoded_layers[num_layer].mean(dim = 1)

In [46]:
def get_embeddings_doc(doc, model, num_layer, window):
    text = doc[0]
    idxs = [i for i,val in enumerate(text) if val=='[MASC]']
    embeddings = []
    for idx in idxs:
        embeddings.append(get_embedding_text(text = text, model = model, num_layer = num_layer, idx =  idx, window = window))
    return embeddings

In [47]:
#t = get_embeddings_doc(docs[0], model = model, num_layer = 23, window = 15)

embeddings_docs = []
for doc in docs:
    embeddings_docs.append((doc[0], doc[1], get_embeddings_doc(doc, model = model, num_layer = 23, window = 15)))

In [48]:
doc = docs[0]

In [49]:
embeddings_docs = []
embeddings_docs.append((doc[0], doc[1], get_embeddings_doc(doc, model = model, num_layer = 23, window = 15)))

In [50]:
#len(embeddings_docs[0][2])

In [43]:
text = (['Once again'], [])

In [44]:
get_embedding_text(text, model, num_layer, idx, window)

NameError: name 'num_layer' is not defined